In [49]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point
import random

weather_csv = gpd.read_file("./기상청_종관기상관측_ASOS.csv", encoding='cp949')
accident_csv = gpd.read_file("제주특별자치도_수난사고 구조출동 현황_2011_2020.csv")
length = len(accident_csv)
idx = int(random.random() * length)
print("coordinate : " + str(accident_csv['loc_info_x'][idx]) + ", " + str(accident_csv['loc_info_y'][idx]))

1547
coordinate : 136983.4, 47159.45


In [2]:
'''
url = 'https://kauth.kakao.com/oauth/token'
rest_api_key = '359e276820f89feab230a8eb1eeadedd'
redirect_uri = 'http://localhost:8888/' # APP에서 등록한 redirect_url
authorize_code = '-Ti1X-TEIwWEg8lHx3xTPOKZk5TdUQMW4M_joY99Vw_GMUHg78lt7g-McQx5fhPTvvUt0worDR4AAAGCFOaPpA'
 
data = {
        'grant_type':'authorization_code',
        'client_id':rest_api_key,
        'redirect_uri':redirect_uri,
        'code': authorize_code,
    }

response = requests.post(url, data=data)
tokens = response.json()
print(tokens["access_token"])
'''

NameError: name 'requests' is not defined

In [91]:
import requests

new_idx = []
new_x = []
new_y = []

for idx in range(0, length):  
    if(str(accident_csv['loc_info_y'][idx]) != ''):
        method = "GET"
        url = "https://dapi.kakao.com/v2/local/geo/transcoord.json"
        params = {'x' : str(accident_csv['loc_info_x'][idx]), 
                  'y' : str(accident_csv['loc_info_y'][idx]),
                  'input_coord' : 'KTM',
                  'output_coord' : 'WGS84'}
        header = {'authorization': 'KakaoAK 359e276820f89feab230a8eb1eeadedd'}

        response = requests.request(method=method, url=url, headers=header, params=params )
        tokens = response.json()

        x = tokens['documents'][0]['x']
        y = tokens['documents'][0]['y']

        new_idx.append(accident_csv['resc_reprt_no'])
        new_x.append(x)
        new_y.append(y)
    else: #has no coordinate
       print(idx)

88


In [68]:
!pip install folium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.0/95.0 kB 3.0 MB/s eta 0:00:00


In [89]:
import folium
m = folium.Map(location = [33.3734235,126.5512051,], zoom_start=8)
m.save('map.html')
m

In [90]:
for i in range(0, len(new_x)):
    folium.Marker(
        location = [new_y[i], new_x[i]],
        icon = folium.Icon(color="red", icon="star")).add_to(m)

m